In [1]:
from keras.applications import ResNet50
from keras.applications import VGG19
from keras.applications import InceptionV3
from keras.applications import Xception
from keras import layers
from keras import optimizers
from keras import models
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow as tf
from keras import regularizers

Using TensorFlow backend.


In [2]:
pretrained_conv_base = Xception(weights = 'imagenet', include_top = False, input_shape = (400,400,3))
pretrained_conv_base.trainable = False
model = models.Sequential()
model.add(pretrained_conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(7, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.RMSprop(lr=1e-5), metrics = ['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 13, 13, 2048)      20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 346112)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               177209856 
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 3591      
Total params: 198,074,927
Trainable params: 177,213,447
Non-trainable params: 20,861,480
_________________________________________________________________


In [3]:
base_dir = '/Users/Danny Han/Desktop/Plastic_Waste_Identifier_Project/Training_Image_DB'
train_dir = base_dir
val_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join(base_dir, "test")


In [5]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

test_datagen = ImageDataGenerator(
rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
train_dir,
target_size = (400, 400),
batch_size = 10,
class_mode = "categorical")

#validation_generator = test_datagen.flow_from_directory(
#val_dir,
#target_size = (400, 400),
#batch_size = 10,
#class_mode = "categorical")


Found 3199 images belonging to 7 classes.


In [6]:
history = model.fit_generator(
train_generator,
steps_per_epoch = 60,
epochs = 100)


Epoch 1/100


c:\users\danny han\appdata\local\programs\python\python36\lib\site-packages\PIL\Image.py:953: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


60/60 [==============================] - 28s 470ms/step - loss: 2.7755 - acc: 0.3767
Epoch 2/100
60/60 [==============================] - 17s 276ms/step - loss: 2.3699 - acc: 0.4850
Epoch 3/100
60/60 [==============================] - 16s 264ms/step - loss: 2.1989 - acc: 0.5283
Epoch 4/100
60/60 [==============================] - 16s 265ms/step - loss: 2.1084 - acc: 0.5383
Epoch 5/100
60/60 [==============================] - 16s 264ms/step - loss: 2.0165 - acc: 0.5600
Epoch 6/100
60/60 [==============================] - 15s 243ms/step - loss: 1.8796 - acc: 0.5996
Epoch 7/100
60/60 [==============================] - 14s 235ms/step - loss: 1.7310 - acc: 0.6250
Epoch 8/100
60/60 [==============================] - 14s 233ms/step - loss: 1.7767 - acc: 0.5783
Epoch 9/100
60/60 [==============================] - 14s 239ms/step - loss: 1.7523 - acc: 0.6150
Epoch 10/100
60/60 [==============================] - 14s 240ms/step - loss: 1.7756 - acc: 0.6217
Epoch 11/100
60/60 [=====================

60/60 [==============================] - 15s 243ms/step - loss: 1.2017 - acc: 0.7550
Epoch 85/100
60/60 [==============================] - 14s 235ms/step - loss: 1.1901 - acc: 0.7583
Epoch 86/100
60/60 [==============================] - 14s 242ms/step - loss: 1.2432 - acc: 0.7680
Epoch 87/100
60/60 [==============================] - 14s 235ms/step - loss: 1.0789 - acc: 0.7783
Epoch 88/100
60/60 [==============================] - 14s 233ms/step - loss: 1.1343 - acc: 0.7533
Epoch 89/100
60/60 [==============================] - 14s 237ms/step - loss: 1.1871 - acc: 0.7283
Epoch 90/100
60/60 [==============================] - 14s 236ms/step - loss: 1.1959 - acc: 0.7533
Epoch 91/100
60/60 [==============================] - 14s 236ms/step - loss: 1.0550 - acc: 0.7698
Epoch 92/100
60/60 [==============================] - 14s 239ms/step - loss: 1.2344 - acc: 0.7417
Epoch 93/100
60/60 [==============================] - 14s 234ms/step - loss: 1.1473 - acc: 0.7600
Epoch 94/100
60/60 [=============

In [11]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('VGG19_acc.jpeg')
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.savefig("VGG19_loss.jpeg")
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [8]:
pretrained_conv_base.trainable = True
trainable = False
for layer in pretrained_conv_base.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    if trainable:
        layer.trainable = True
    else:
        
        layer.trainable = False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 12, 12, 512)       20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 73728)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               37749248  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 3591      
Total params: 48,337,991
Trainable params: 37,752,839
Non-trainable params: 10,585,152
_________________________________________________________________


c:\users\danny han\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [9]:
model.compile(loss='categorical_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-5),
metrics=['acc'])

In [10]:
history = model.fit_generator(
train_generator,
steps_per_epoch=60,
epochs=100,
validation_data=validation_generator,
validation_steps=50)

Epoch 1/100
60/60 [==============================] - 20s 334ms/step - loss: 1.3663 - acc: 0.5630 - val_loss: 1.4373 - val_acc: 0.5855
Epoch 2/100
60/60 [==============================] - 16s 274ms/step - loss: 1.3176 - acc: 0.6150 - val_loss: 1.2581 - val_acc: 0.6480
Epoch 3/100
60/60 [==============================] - 17s 280ms/step - loss: 1.2248 - acc: 0.6217 - val_loss: 1.2187 - val_acc: 0.6781
Epoch 4/100
60/60 [==============================] - 17s 275ms/step - loss: 1.2466 - acc: 0.6296 - val_loss: 1.3134 - val_acc: 0.6439
Epoch 5/100
60/60 [==============================] - 17s 276ms/step - loss: 1.1105 - acc: 0.6667 - val_loss: 1.2963 - val_acc: 0.6320
Epoch 6/100
60/60 [==============================] - 17s 280ms/step - loss: 1.1764 - acc: 0.6600 - val_loss: 1.3912 - val_acc: 0.6439
Epoch 7/100
60/60 [==============================] - 16s 271ms/step - loss: 1.1157 - acc: 0.6815 - val_loss: 1.2634 - val_acc: 0.6559
Epoch 8/100
60/60 [==============================] - 16s 275ms

Epoch 62/100
60/60 [==============================] - 17s 277ms/step - loss: 0.6287 - acc: 0.8600 - val_loss: 1.3249 - val_acc: 0.7364
Epoch 63/100
60/60 [==============================] - 16s 273ms/step - loss: 0.6539 - acc: 0.8500 - val_loss: 1.4402 - val_acc: 0.6840
Epoch 64/100
60/60 [==============================] - 16s 274ms/step - loss: 0.6425 - acc: 0.8331 - val_loss: 1.3469 - val_acc: 0.7103
Epoch 65/100
60/60 [==============================] - 16s 271ms/step - loss: 0.6427 - acc: 0.8417 - val_loss: 1.3658 - val_acc: 0.7304
Epoch 66/100
60/60 [==============================] - 17s 282ms/step - loss: 0.6544 - acc: 0.8317 - val_loss: 1.3835 - val_acc: 0.7160
Epoch 67/100
60/60 [==============================] - 17s 279ms/step - loss: 0.6203 - acc: 0.8433 - val_loss: 1.3466 - val_acc: 0.7163
Epoch 68/100
60/60 [==============================] - 17s 279ms/step - loss: 0.5990 - acc: 0.8650 - val_loss: 1.3601 - val_acc: 0.6922
Epoch 69/100
60/60 [==============================] - 1

In [26]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(400, 400, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu', kernel_regularizer = regularizers.l2(0.001)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(7, activation='softmax'))

In [8]:
model.save('Xception_Trial_4.h5')

In [7]:
pretrained_conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 199, 199, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 199, 199, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 199, 199, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con